# Probar la búsqueda en Azure AI Search

### Generar los embedings para una consulta

In [23]:
#%pip install azure-search-documents
#%pip install azure-identity

import os
import dotenv
import openai
import json

from azure.identity import DefaultAzureCredential, get_bearer_token_provider
from azure.search.documents import SearchClient
from azure.search.documents.models import (
    VectorizedQuery,
    QueryType,
    QueryCaptionType,
    QueryAnswerType,
)


In [8]:
openai_client = openai.OpenAI(
    base_url="https://models.inference.ai.azure.com",
    api_key=os.environ["GITHUB_TOKEN"]
)
MODEL_NAME = "text-embedding-3-small"

In [11]:
query_input = "Resume la hoja de vida de Diego Zumárraga Mera"
embeddings_response = openai_client.embeddings.create(
    model=MODEL_NAME,
    input=query_input,
)
query_embedding = embeddings_response.data[0].embedding
print(query_embedding)

[0.04746444150805473, 0.07716769725084305, 0.013239097781479359, 0.010300321504473686, -0.07539157569408417, -0.005494867917150259, -0.0009596300660632551, 0.09721914678812027, -0.011153326369822025, -0.016931556165218353, 0.0010852437699213624, -0.02112646773457527, 0.0010107519337907434, -0.018754415214061737, 0.040219746530056, 0.0629120022058487, -0.027015704661607742, 0.03003043308854103, 0.031853292137384415, 0.04199586808681488, 0.011170853860676289, 0.00023388244153466076, -0.012643163092434406, 0.011094900779426098, 0.022341707721352577, -0.009347994811832905, 0.012666532769799232, 0.015576096251606941, 0.03961212933063507, -0.055106427520513535, -0.026151014491915703, -0.042323045432567596, 0.031900033354759216, 0.020729178562760353, -0.04440297558903694, -0.01840386539697647, -0.004300077445805073, 0.01998133957386017, -0.034657690674066544, -0.00618720380589366, 0.033185381442308426, 0.008565099909901619, 0.03813982009887695, 0.055760789662599564, 0.01728210598230362, 0.026

### Buscar la consulta en Azure AI Search

In [12]:
item = {"query": query_input,
        "embedding": query_embedding
}

In [14]:
search_client = SearchClient(
        endpoint=os.environ["AZURE_AI_SEARCH_ENDPOINT"],
        index_name="my-index",
        credential=DefaultAzureCredential(),
    )

In [37]:
results = []
vector_query = VectorizedQuery(
            vector=item["embedding"], k_nearest_neighbors=3, fields="embedding"
        )
result = search_client.search(
    search_text=item["query"],
    vector_queries=[vector_query],
    query_type=QueryType.SEMANTIC,
    semantic_configuration_name="default",
    query_caption=QueryCaptionType.EXTRACTIVE,
    query_answer=QueryAnswerType.EXTRACTIVE,
    top=2,
)

### Obtener la mejor respuesta

In [38]:
answers = result.get_answers()
if answers:
    print("Answers:")
    for answer in answers:
        print(f"Answer: {answer.text}")
        print(f"Confidence: {answer.score}")

Answers:
Answer: Hoja de Vida: Diego Zumárraga Mera. Ingeniero de software con experiencia en desarrollo de aplicaciones web y móviles, apasionado por la inteligencia artificial y el aprendizaje automático.
Confidence: 0.9860000014305115


### Obtener todos los resultados

In [39]:
docs = [
        {
            "id": doc["id"],
            "title": doc["title"],
            "content": doc["content"],
            "rerankerScore": doc["@search.reranker_score"],
        }
        for doc in result
    ]

# Remove duplicates
results.extend([i for i in docs if i["id"] not in [x["id"] for x in results]])

print(f"Found {len(results)} results")
for i, doc in enumerate(results):
    print(f"Result {i + 1}:")
    print(f"Title: {doc['title']}")
    print(f"Content: {doc['content']}")
    print(f"Reranker Score: {doc['rerankerScore']}")
    print()

Found 2 results
Result 1:
Title: Diego Zumárraga Mera
Content: Hoja de Vida: Diego Zumárraga Mera. Ingeniero de software con experiencia en desarrollo de aplicaciones web y móviles, apasionado por la inteligencia artificial y el aprendizaje automático.
Reranker Score: 2.8379650115966797

Result 2:
Title: Lionel Messi
Content: Hoja de vida: Lionel Messi. Futbolista argentino, considerado uno de los mejores jugadores de fútbol de todos los tiempos.
Reranker Score: 0.8146551847457886

